In [1]:
## database imports
from database.market import Market
from database.adatabase import ADatabase

## data processor import
from processor.processor import Processor as p

## modeler import
from modeler_strats.universal_modeler import UniversalModeler

## strategy impors
from strategy.speculation import Speculation as spec
from strategy.classification import Classification as classer

## miscellaneous
import numpy as np
import matplotlib.pyplot as plt
import pickle
import math
from statistics import variance
from datetime import datetime, timedelta
import pytz
import pandas as pd
from tqdm import tqdm

In [2]:
start_date = datetime(2023,1,1)
end_date = datetime.now()

In [3]:
market = Market()
speculation_db = ADatabase("btc_spec")
classification_db = ADatabase("btc_spec_classification")
current_db = ADatabase("btc_current")
umod = UniversalModeler()

In [4]:
market.connect()
prices = market.retrieve("crypto")
sp500 = market.retrieve("sp500")
sp500 = sp500.rename(columns={"Symbol":"ticker"})
market.disconnect()
current_prices = pd.read_csv("./csv_files/prices/BTC-USD.csv")
current_prices["ticker"] = "BTC"
current_prices = p.column_date_processing(current_prices)
prices = p.column_date_processing(prices)
prices["ticker"] = "BTC"
prices = pd.concat([prices,current_prices])

In [5]:
factors = [str(x) for x in range(14)]
included_columns=[
    "year",
    "week",
    "ticker",
    "adjclose",
]
included_columns.extend(factors)

In [6]:
classification_factors = ["d1","d2","d3","rolling14"]
included_columns.extend(classification_factors)

In [7]:
ticker_data = prices.copy()
ticker_data = spec.transform(ticker_data)
ticker_data = classer.transform(ticker_data)
ticker_data.dropna(inplace=True)
ticker_data["ticker"] = "BTC"
ticker_data = ticker_data[included_columns]
data = ticker_data.copy().dropna()

In [8]:
prediction_slice = data[data["year"]>=start_date.year-2].reset_index(drop=True)

In [9]:
speculation_db.connect()
models = speculation_db.retrieve("models")
speculation_db.disconnect()

In [10]:
simulation = umod.recommend(models,prediction_slice.copy(),factors)
simulation["week"] = simulation["week"] + 1
simulation["prediction"] = (simulation["cat_prediction"] + simulation["xgb_prediction"] + simulation["tf_prediction"]) / 3
speculation_db.connect()
speculation_db.drop("predictions")
speculation_db.store("predictions",simulation)
speculation_db.disconnect()

Keras model archive loading:
File Name                                             Modified             Size
config.json                                    2023-03-03 11:19:38         2522
metadata.json                                  2023-03-03 11:19:38           64
variables.h5                                   2023-03-03 11:19:38        84160
Keras weights file (<HDF5 file "variables.h5" (mode r)>) loading:
...layers\dense
......vars
.........0
.........1
...layers\dense_1
......vars
.........0
.........1
...layers\dense_2
......vars
.........0
.........1
...metrics\mean
......vars
.........0
.........1
...metrics\mean_absolute_percentage_error
......vars
.........0
.........1
...optimizer
......vars
.........0
.........1
.........10
.........11
.........12
.........2
.........3
.........4
.........5
.........6
.........7
.........8
.........9
...vars
4/4 [==============================] - 0s 1ms/step


In [11]:
classification_db.connect()
classification_models = classification_db.retrieve("models")
classification_db.disconnect()

In [12]:
classification_simulation = umod.recommend(classification_models,prediction_slice.copy(),classification_factors)
classification_simulation["week"] = classification_simulation["week"] + 1
classification_simulation["classification_prediction"] = ((classification_simulation["cat_prediction"] + \
                                                          classification_simulation["xgb_prediction"] + \
                                                          classification_simulation["tf_prediction"]) / 3) > 0.5
classification_simulation["classification_prediction"] = [int(x) for x in classification_simulation["classification_prediction"]]
classification_db.connect()
classification_db.drop("predictions")
classification_db.store("predictions",classification_simulation)
classification_db.disconnect()

Keras model archive loading:
File Name                                             Modified             Size
config.json                                    2023-03-30 10:52:04         2149
metadata.json                                  2023-03-30 10:52:04           64
variables.h5                                   2023-03-30 10:52:04        23208
Keras weights file (<HDF5 file "variables.h5" (mode r)>) loading:
...layers\dense
......vars
.........0
.........1
...layers\dense_1
......vars
.........0
.........1
...layers\dropout
......vars
...metrics\mean
......vars
.........0
.........1
...metrics\mean_metric_wrapper
......vars
.........0
.........1
...optimizer
......vars
.........0
.........1
.........2
.........3
.........4
.........5
.........6
.........7
.........8
...vars
4/4 [==============================] - 0s 1ms/step
